# Raw Data Loading, Merging and Filter

### Get file names and prepare for merging

In [34]:
import pandas as pd
pd.set_option('display.max_rows', 500)
import os

In [35]:
PATH = "../00_data/0_raw/"
all_files = [f for f in os.listdir(PATH)] #get all file names
sorted_files = sorted(os.listdir(PATH))

#use file names as column names for later
def new_col_names(name):
    #print(os.path.basename(name).split('.')[0].replace('-', '_'))
    return os.path.basename(name).split('.')[0].replace('-', '_')

#make label u5mr first (child-mortality at position 1)
label_file = sorted_files.pop(1)
sorted_files.insert(0, label_file)

### Load and merge all csv files and exclude non-countries

In [36]:
big_df = None
joins = ['Entity', 'Code', 'Year']

EXCLUDE_NO_COUNTRIES = ["Africa", "Asia", "Europe", "European Union (27)", "High-income countries", "Low-income countries", "Lower-middle-income countries", 
                      "North America", "Oceania", "South America", "Upper-middle-income countries", "World"]

#change column names and add as columns to df
for name in sorted_files:
    cols_names = new_col_names(name)

    df = pd.read_csv(os.path.join(PATH, name), usecols=[0, 1, 2, 3])
    df.columns = joins + [cols_names]

    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html
    df = df[~df['Entity'].isin(EXCLUDE_NO_COUNTRIES)] #remove continents etc. from df
    
    df = df.set_index(joins) #entity, code, year as index

    if big_df is None:
        big_df = df.copy() 
    else: #outer left join - merge all dfs
        big_df = big_df.merge(
            df, 
            left_index=True, 
            right_index=True, 
            how='left' 
        )

#big_df.loc[("Ghana")].head(15)
big_df.info()
#big_df.shape

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13038 entries, ('Afghanistan', 'AFG', np.int64(1957)) to ('Zimbabwe', 'ZWE', np.int64(2023))
Data columns (total 10 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   child_mortality_igme                      13038 non-null  float64
 1   annual_healthcare_expenditure_per_capita  4357 non-null   float64
 2   gdp_per_capita_worldbank                  6299 non-null   float64
 3   nurses_and_midwives_per_1000_people       3109 non-null   float64
 4   physicians_per_1000_people                4986 non-null   float64
 5   prevalence_of_undernourishment            3661 non-null   float64
 6   share_of_population_urban                 11580 non-null  float64
 7   share_without_improved_water              4433 non-null   float64
 8   vaccination_coverage_who_unicef           7897 non-null   float64
 9   years_of_schooling                    

### Test: Find 6 years period with least NaN values

In [37]:
# which 6 year span has least NaN values for whole df, start from 2000 => 'cause where MDG goals started (most current years)
# between 2000 and 2019 find the 6 year period to use for later as main df, after can be more biased cause of Corona period
# so i can have a argument for my thesis in "Methodik"
# Zeitraum: 2000 - 2019, loop in 6 year periods (2000-2005, 2001-2006, 2002-2007 ... 2013-2018...)
# Note: auch von 1960 aus betrachtet ist 2013-2018 der Zeitraum mit den wenigsten Null Values 
nan_count = 0
df_period = big_df.copy()
year_val = df_period.index.get_level_values(2)

for year_start in range(2000,2019):
    year_end = year_start + 5
    # get filtered df for each period 
    df_filtered = df_period[(year_val >= year_start) & (year_val <= year_end)]
    # get number of all NaNs and print
    nan_count = df_filtered.isna().sum().sum()
    print(f"From {year_start} - {year_end}, NaN values count: {nan_count}")

From 2000 - 2005, NaN values count: 2003
From 2001 - 2006, NaN values count: 1813
From 2002 - 2007, NaN values count: 1763
From 2003 - 2008, NaN values count: 1672
From 2004 - 2009, NaN values count: 1586
From 2005 - 2010, NaN values count: 1590
From 2006 - 2011, NaN values count: 1562
From 2007 - 2012, NaN values count: 1524
From 2008 - 2013, NaN values count: 1479
From 2009 - 2014, NaN values count: 1463
From 2010 - 2015, NaN values count: 1447
From 2011 - 2016, NaN values count: 1423
From 2012 - 2017, NaN values count: 1368
From 2013 - 2018, NaN values count: 1273
From 2014 - 2019, NaN values count: 1285
From 2015 - 2020, NaN values count: 1318
From 2016 - 2021, NaN values count: 1396
From 2017 - 2022, NaN values count: 1555
From 2018 - 2023, NaN values count: 2360


In [38]:
# 2013 - 2018 has least NaN vals, take as main starting dataframe
df_main_period = df_period[(year_val >= 2013) & (year_val <= 2018)]
df_main_period.isna().sum().sum()

np.int64(1273)

### Code & Year as MultiIndex

In [39]:
#turn Entity into col
df_main_period = df_main_period.reset_index(level=0)
df_main_period.head()

Entity  child_mortality_igme  \
Code Year                                      
AFG  2013  Afghanistan              7.802299   
     2014  Afghanistan              7.512363   
     2015  Afghanistan              7.243871   
     2016  Afghanistan              6.997004   
     2017  Afghanistan              6.763737   

           annual_healthcare_expenditure_per_capita  gdp_per_capita_worldbank  \
Code Year                                                                       
AFG  2013                                 177.62009                 3046.5798   
     2014                                 201.79086                 3017.9426   
     2015                                 215.22618                 2967.6921   
     2016                                 238.74481                 2958.7854   
     2017                                 264.20660                 2952.9990   

           nurses_and_midwives_per_1000_people  physicians_per_1000_people  \
Code Year                                                                    
AFG  2013                                0.255                       0.291   
     2014                                0.151                       0.304   
     2015                                0.133                       0.291   
     2016                                0.151                       0.284   
     2017                                0.179                       0.245   

           prevalence_of_undernourishment  share_of_population_urban  \
Code Year                                                              
AFG  2013                            19.4                     24.373   
     2014                            19.3                     24.587   
     2015                            20.0                     24.803   
     2016                            20.5                     25.020   
     2017                            21.4                     25.250   

           share_without_improved_water  vaccination_coverage_who_unicef  \
Code Year                                                                  
AFG  2013                     38.863914                             64.0   
     2014                     36.337383                             62.0   
     2015                     33.812515                             64.0   
     2016                     31.289620                             66.0   
     2017                     28.765442                             64.0   

           years_of_schooling  
Code Year                      
AFG  2013            0.672197  
     2014            0.710609  
     2015            0.749020  
     2016            1.009183  
     2017            1.269347

In [40]:
#Scale U5MR uo to 1000, scale used intrnational by UN IGME & co. 
df_main_period["child_mortality_igme"] = df_main_period["child_mortality_igme"] * 10
df_main_period.head()

Entity  child_mortality_igme  \
Code Year                                      
AFG  2013  Afghanistan             78.022990   
     2014  Afghanistan             75.123625   
     2015  Afghanistan             72.438707   
     2016  Afghanistan             69.970040   
     2017  Afghanistan             67.637370   

           annual_healthcare_expenditure_per_capita  gdp_per_capita_worldbank  \
Code Year                                                                       
AFG  2013                                 177.62009                 3046.5798   
     2014                                 201.79086                 3017.9426   
     2015                                 215.22618                 2967.6921   
     2016                                 238.74481                 2958.7854   
     2017                                 264.20660                 2952.9990   

           nurses_and_midwives_per_1000_people  physicians_per_1000_people  \
Code Year                                                                    
AFG  2013                                0.255                       0.291   
     2014                                0.151                       0.304   
     2015                                0.133                       0.291   
     2016                                0.151                       0.284   
     2017                                0.179                       0.245   

           prevalence_of_undernourishment  share_of_population_urban  \
Code Year                                                              
AFG  2013                            19.4                     24.373   
     2014                            19.3                     24.587   
     2015                            20.0                     24.803   
     2016                            20.5                     25.020   
     2017                            21.4                     25.250   

           share_without_improved_water  vaccination_coverage_who_unicef  \
Code Year                                                                  
AFG  2013                     38.863914                             64.0   
     2014                     36.337383                             62.0   
     2015                     33.812515                             64.0   
     2016                     31.289620                             66.0   
     2017                     28.765442                             64.0   

           years_of_schooling  
Code Year                      
AFG  2013            0.672197  
     2014            0.710609  
     2015            0.749020  
     2016            1.009183  
     2017            1.269347

### Save merged df as 'first step' raw csv file

In [41]:
#save main_period as csv
df_main_period.to_csv('../00_data/1_interim/merged_data.csv', index=True)